<a name="top"></a>
## Table of content

* [Цели и задачи данного ноутбука](#purpose)
* [Рисование свечного графика](#candle-chart)
* [Таблица корреляции](#correlation)
* [Сохранение графика в картинку, редактирование картинки](#pillow)

<a name="purpose"></a>
## Charting and researching with plotly numpy

[Наверх](#top)

Цели для данного ноутбука:

1. Научиться строить графики котировок (свечи, линии) (+)
1. Подключить модуль расчета индикаторов TA Lib и научиться дополнять df этими данными (+)
1. Попробывать базовые элементы технического анализа
    1. Корреляция между индексом и акцией (+)
    1. Две MA на графике, их анализ (+)
    1. Поиск и подсветка отдельных паттернов. Для примера берем молот из TA Lib (+)
    
Импортируем необходимые модули и подгружаем датасеты.

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime

ticker = 'GAZP'
df = pd.read_csv('../data/moex/' + ticker + '_processeddata.csv')
df.date_time = pd.to_datetime(df.date_time)
df = df.set_index('date_time')

<a name="candle-chart"></a>
## Основная функция для построения графиков
[Наверх](#top)

Описание функции. Функция принимает в себя следующие параметры:
* **ticker** - *str* Текстовое название тикера, по которому строится график.
* **df_price** - *DataFrame* Датафрейм с котировками для построения свечного графика. Датасет обязательно должен содержать колонки open, high, low, close. Дата котировки должна быть в индексе, т.е. должна была быть сделана операция: df = df.set_index('date_time') 
* **on_price_plots** *list* список, состоящий из словарей, описывающий какие индикаторы должны быть добавлены на график цены. Ключи необходимо указать следующие:
    * 'series' *Series* Вектор значений индикатора, который необходимо добавить на график. Длина вектора должна соответвовать длине окна отображаемого графика.
    * 'name' *str* Имя индикатора для отображения в легенде
    * 'color' *str* Цвет индикатора
* **subplots** *list* список, состоящий из словарей, описывающий какие индикаторы должны быть отображены в отдельных окнах. Ключи необходимо указать следующие:
    * 'chart_type' *str* Тип графика индикатора. Может принимать следующие значения 'line' - линия, 'bar' - гистограмма.
    * 'series' *Series* Вектор значений индикатора. Длина вектора должна соответвовать длине окна отображаемого графика.
    * 'name' *str* Имя индикатора для отображения в легенде 
    * 'color' *str* Цвет индикатора
* **holydays** *list* список праздничных дней для данного тикера, что не добавлял разрывов на график.
* **patterns** *list* список, состоящий из словарей, описывающий к каким паттернам необходимо добавить подписи на графике. Ключи необходимо указать следующие:
    * 'pattern_name' *str* текст для отображения. 
    * 'pattern_series' *Series* Вектор значений паттернов, которые нужно отобразить. Дата обязательно должна быть в индексе.
    * 'anchor' *str* Показывает сверху 'up' или снизу 'down' бара выводить текст подписи.
    * 'color' *str* Цвет текста и стрелки
* **lines** *list* список, состоящий из словарей, описывающий линии, как горизонтальные, так и наклонные, которые необходимо отрисовать на графике. Ключи необходимо указать следующие:
    * 'x_series' *Series* Вектор значений дат, относительно которых нужно выводить линию.
    * 'y_series' *Series* Вектор значений линии относительно оси y. Размерность должна соответствовать 'x_series'.
    * 'text' *str* текст для отображения. 
    * 'color' *str* Цвет линии и текста.
    * 'opacity': *float* Толщина линии.
* **is_show_figs** *bool* указывает показать ли график или нет.

Функция возращает объект графика для дальнейших манипуляций, напр. сохранение в файл.

In [3]:
def plot_candle_chart(ticker, df_price, on_price_plots=[], subplots=[], holydays=[], patterns=[], lines=[], is_show_figs=False):
    if len(subplots) == 0:
        row_heights = [1]
    elif len(subplots) == 1:
        row_heights = [0.7, 0.3]
    else:
        row_heights = [0.5]
        delta = 0.5/len(subplots)
        for i in range(0, len(subplots)):
            row_heights.append(delta)

    fig = make_subplots(rows=len(subplots)+1, 
                        cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing=0.02, 
                        row_heights=row_heights)

    fig.add_trace(go.Candlestick(x=df_price.index,
                                 open=df_price['open'],
                                 high=df_price['high'],
                                 low=df_price['low'],
                                 close=df_price['close'],
                                 name=ticker,
                                 increasing={'line': {'width': 1, 'color': 'green'},
                                             'fillcolor': 'green'},
                                 decreasing={'line': {'width': 1, 'color': 'red'},
                                             'fillcolor': 'red'}),
                  row=1, col=1)
    fig.update_yaxes(title_text="График цены", row=1, col=1)
    
    
    # Рисуем линии на графике цены
    for pp in on_price_plots:
        fig.add_trace(go.Scatter(x=df_price.index,
                                     y=pp['series'],
                                     name=pp['name'],
                                     line_color=pp['color'],
                                     opacity=1),
                          row=1, col=1)  
    # Рисуем subplots    
    row_num = 2
    for sp in subplots:
        if sp['chart_type'] == 'line':
            fig.add_trace(go.Scatter(x=df_price.index,
                                     y=sp['series'],
                                     name=sp['name'],
                                     line_color=sp['color'],
                                     opacity=1),
                          row=row_num, col=1)
        elif sp['chart_type'] == 'bar':
            fig.add_trace(go.Bar(x=df_price.index,
                                 y=sp['series'],
                                 name=sp['name'],
                                 marker_color=sp['color']),
                          row=row_num, col=1)
        else:
            raise ValueError('Unknown subplot chart_type "{0}"'.format(sp['chart_type']))
        fig.update_yaxes(title_text=sp['name'], row=row_num, col=1)
        row_num += 1
    
    # Добавляем аннотации на график
    annotations = []
    up_px = 20
    for p in patterns:
        for index, row in p['pattern_series'].iterrows():
            annotations.append(
                dict(x=index, 
                     y=row['high'] if p['anchor']=='up' else row['low'], 
                     ax=-10, 
                     ay=up_px if p['anchor']=='down' else up_px*-1, 
                     xref='x', 
                     yref='y', 
                     showarrow=True, 
                     arrowhead=1, 
                     arrowcolor=p['color'], 
                     xanchor='center', 
                     text=p['pattern_name'], 
                     font=dict(
                            family="Arial, monospace",
                            size=10,
                            color=p['color']
                            )
                     ))
        up_px += 10
        
    fig.update_layout(annotations=annotations)
    
    # Рисуем линии, как наклонные, так и горизонтальные
    for l in lines:
        fig.add_trace(go.Scatter(x=l['x_series'],
                                 y=l['y_series'],
                                 mode='lines',
                                 name=l['text'],
                                 line_color=l['color'],
                                 opacity=l['opacity']),
                          row=1,
                          col=1)
        
        fig.add_annotation(x=l['x_series'][0],
                            y=l['y_series'][0],
                            text=l['text'],
                            showarrow=False,
                            xshift=-40,
                            font=dict(
                                    family="Arial, monospace",
                                    size=10,
                                    color=l['color']
                                    ))
        
    
    # Финальные обработки графика
    # Замещаем функцию fig = pts_layout(fig, holydays=holydays, is_show_figs=is_show_figs)
    if len(holydays) > 0:
        fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"]), holydays])
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.update_layout(height=800,
                      width=1000,
                      xaxis_rangeslider_visible=False,
                      template='plotly_white',
                      )
    
    if is_show_figs:
        fig.show()
    
    return fig

In [6]:
#df = df.set_index('date_time')
holydays = dict(values=["2020-02-24", "2020-03-09", "2020-05-01", "2020-05-11", "2020-06-12", "2020-06-24", "2020-07-01", '2020-11-04'])
window = 80
on_price_plots = [
                    {'series': df['MA_slow'][-window:], 'name': 'Slow MA', 'color': 'blue'},
                ]
subplots = [
                {'chart_type': 'line', 'series': df['CCI'][-window:], 'name': 'CCI', 'color': 'blue'},
                {'chart_type': 'bar', 'series': df['MACD'][-window:], 'name': 'MACD', 'color': 'red'},
                {'chart_type': 'line', 'series': df['WILLR'][-window:], 'name': 'WILLR', 'color': 'black'},
           ]
df_for_patterns = df[-window:]
patterns = [
            {'pattern_name': 'ShootingStar', 'pattern_series': df_for_patterns[df_for_patterns['ShootingStar'] < 0], 'anchor': 'up', 'color': 'black'},
            {'pattern_name': 'Hummer', 'pattern_series': df_for_patterns[df_for_patterns['Hummer'] > 0], 'anchor': 'down', 'color': 'pink'},
            {'pattern_name': 'DivBar', 'pattern_series': df_for_patterns[df_for_patterns['DivBar'] > 0], 'anchor': 'down', 'color': 'green'},
            {'pattern_name': 'DivBar1', 'pattern_series': df_for_patterns[df_for_patterns['DivBar'] < 0], 'anchor': 'up', 'color': 'red'},
            ]
#np.full(20, 150)
lines = [{'x_series': df.index.values[-30:-20], 'y_series': np.full(20, 150), 'text': 'support line', 'color': 'lime', 'opacity': 1},
         {'x_series': df.index.values[-30:-25], 'y_series': [190, 195, 200, 205, 210], 'text': 'support line', 'color': 'lime', 'opacity': 0.8}]
plot_candle_chart('GAZP', df[-window:], on_price_plots=on_price_plots, subplots=subplots, patterns=patterns, lines=lines, holydays=holydays, is_show_figs=True)

<a name="correlation"></a>
## Исследуем корреляцию между двумя тикерами
[Наверх](#top)

Возьмем для примера индекс и акцию. Попробуем проверить корреляцию между Close этих тикеров, а также между MA3 по ним. Дозагрузим дополнительные данные.

In [11]:
import numpy as np

df_moex = pd.read_csv('../data/moex/' + 'SBER' + '_processeddata.csv')
df_gazp = pd.read_csv('../data/moex/' + 'GAZP' + '_processeddata.csv')

df_moex['ma3'] = ta.MA(df_moex['close'], timeperiod=8, matype=0)
df_gazp['ma3'] = ta.MA(df_gazp['close'], timeperiod=8, matype=0)

corr_coef = np.corrcoef(x=df_moex['close'][-500:], y=df_gazp['close'][-500:])

fig = go.Figure(data=go.Heatmap(
                   z=corr_coef,
                   x=['MOEX_data', 'GAZP_data'],
                   y=['MOEX_data', 'GAZP_data'],
                   hoverongaps = False))
fig.show()

df_moex = df_moex.dropna(subset=['ma3'])
df_gazp = df_gazp.dropna(subset=['ma3'])
corr_coef = np.corrcoef(x=df_moex['ma3'][-500:], y=df_gazp['ma3'][-500:])
fig = go.Figure(data=go.Heatmap(
                   z=corr_coef,
                   x=['MOEX_data', 'GAZP_data'],
                   y=['MOEX_data', 'GAZP_data'],
                   hoverongaps = False))
fig.show()

<a name="pillow"></a>
## Pillow - Работа с изображениями
[Наверх](#top)

Делаем уменьшение размера изображения, а также наносим водяной знак. А затем сохраняем изображение. 

TODO Добавить функцию saveimg

In [17]:
from PIL import Image, ImageDraw, ImageFont

image = Image.open('ALRS_channel.png')
image.show()

image2 = Image.open('FIVE_daily.png')
#image2.show()

 
image2_resized = image.resize((638, 510), Image.ANTIALIAS)
#image2_resized.show()
#image2_resized.save('FIVE_daily.png', 'png')

# watermark
# make the image editable
drawing = ImageDraw.Draw(image)
drawing.text((70, 80), 
             'www.pandoratradingsolutions.ru', 
             fill=(160, 160, 160), 
             font=ImageFont.truetype("Arial.ttf", 10))
image.show()
#image.save('ALRS_channel.png', 'png')

OSError: cannot open resource